#  Install package

In [202]:
# ! pip install pyvis
# ! pip install networkx

# Import package

In [203]:
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
import openai


# Define Solution class
Please run the following cell to define the functions

In [204]:
# %load_ext autoreload
# %autoreload 2

import LLM_Geo_Constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from LLM_Geo_kernel import Solution

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Demonstration Cases

## Input task and data desciption

In [205]:
# print(DATA_LOCATIONS)

In [224]:
# Case 1: population living near hazardous waster
'''
TASK = r"""1) Find out the total population that lives within a tract that contain hazardous waste facilities. The study area is North Carolina, US.
2) Generate a map to show the spatial distribution of population at the tract level and highlight the borders of tracts that have hazardous waste facilities.
"""

DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file location: https://github.com/gladcolor/LLM- Geo/raw/master/overlay_analysis/Hazardous_Waste_Sites.zip.",
                  "NC tract boundary shapefile location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_shp_37.zip. The tract id column is 'Tract'.",
                  "NC tract population CSV file location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv. The population is stored in 'TotalPopulation' column. The tract ID column is 'GEOID'."
                 ]
'''

# Case 2: mobility data retrieval and visulization
"""
TASK = r'''
1) Show the monthly change rates of each administrative regions in a France map. Each month is a sub-map in a map matrix. The base of the change rate is January 2020. 
2) Draw a line chart to show the monthly change rate trends of all administrative regeions.

'''

DATA_LOCATIONS = ["ESRI shapefile for France administrative regions:" + \
                  "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip." + \
                  "The 'GID_1' column is the administrative region code, 'NAME_1' column is the administrative region name.",
                  "REST API url with parameters for mobility data access:" + \
                  "http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020." + \
                  "The response is in CSV format. There are three columns in the response: " + \
                  "place,date (format:2020-01-07), and intra_movement. 'place' column is the administractive region code, France administrative regions start with 'FRA'.",
                 ]

# Bug: 
# base_month =  "2020-01" # wrong: pd.to_datetime("2020-01")
# print(region_monthly[region_monthly['month_year'] == base_month])

"""

# Case 3: mobility data retrieval and visulization
# TASK = r"""1) Find out the Autism service providers' addresses or location from their website. The address usually listed in the homepage, 'about' or 'contact' page. The latter two pages usually contain 'about' or 'contact' in links embedded in the homepage.
# 2) A provider may have multiple service address. If cannot find the address, simply return an empty text, DO NOT make up fake addresses. 
# 3) You need to send webpage text ChatGPT to extract address. Use this pre-written function your designed detailed prompt to get response from ChatGPT: helper.get_LLM_reply(prompt=your_prompt_with_webpage_text, model=r"gpt-4",). Use this statement to extract content from the returned response: response['choices'][0]['message']['content']. Let ChatGPT reply in json format as {'address': extracted_address}.  DO NOT reuturn explaination or conversation, return the address or empty text only. 
# 4) Save the extracted addresses as "Address" column, together with the given 'Provider' and 'Web Site' columns. If there are multile addresses for a provider, each address is a row in the CSV file.
# """

# DATA_LOCATIONS = ["Autism service provider webpage file location: E:\Research\LLM-Geo\Address_extraction\ACE_providers_AGIS.csv. The 'Web Site' column is the URL, the 'Provider' column is the provider name.",                  
#                  ]

# Case 4: COVID-19 prevalence 
TASK = r"""1) Draw map matrix of South Carolina counties' monthly COVID-19 infection ratio with a weekly smooth in 2021.
2) the infection ratio = (infection of this month / county popultion).
"""

API_DOC_LOCATION = [(1, r'https://raw.githubusercontent.com/gladcolor/LLM-Geo/master/COVID-19/CensusData_API_DOC.txt')] 
# [(Input_data_index, API_cocumentation_path)]

DATA_LOCATIONS = ["COVID-19 data case in 2021 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2021.csv. It is a CSV file; there are 5 columns: date (format: 2021-02-01),county,state,fips,cases,deaths",                  
                  "Population data: use Python library CensusData to obtain data. ",
                 ]

# add the API documentation to DATA_LOCATION
for idx, path in API_DOC_LOCATION:
    with open(path, 'r') as f:
        docs = f.readlines(f)
    DATA_LOCATIONS += "The documentation is: \n" + docs


# https://raw.githubusercontent.com/gladcolor/LLM-Geo/master/Address_extraction/ACE_providers_AGIS.csv

# TASK = r'''1)Retrieve the data from the REST API and plot the intra_movement column of the returned data as line chart to show the temporal trend of all states. 
# 2) plot the temporal trend of the movement for each state. Each state figure will be sub figure in the plot. The plot has 5 columns. In addition, please add a weekly smoothed line to each sub plot, and change the line color to orange.
# 3) Using the REST API with date range from 01/01/2020 to 12/31/2020 to analyze the movement reduction rate for each state during two periods: the first period is 01/01/2020-02/29/2020, second period is 03/01/2020 to 04/30/2020. Please find out the reduction rate for each state during the two periods, and create a table to report the result with two columns: state name, reduction rate, sorted by reduction rate.
# '''
# '''
# DATA_LOCATIONS = ["REST API url with parameters for data access: http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=us_state&begin=01/01/2020&end=12/31/2020; The response is in CSV format. There are three columns in the response: place,date,intra_movement; place refers to the state name."
#                  ]
# '''
# 3) Show the administrative region name in the map and chart.
# 
# task_name ='Resident_at_risk_counting'
# task_name ='France_mobility_changes_2020'  
task_name ='Address_extraction'  

save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
# model=r"gpt-3.5-turbo"
model=r"gpt-4"
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
print(solution.graph_prompt)

OSError: [Errno 22] Invalid argument: 'https://raw.githubusercontent.com/gladcolor/LLM-Geo/master/COVID-19/CensusData_API_DOC.txt'

## Get graph code from GPT API

In [207]:
response_for_graph = solution.get_LLM_response_for_graph() 
solution.graph_response = response_for_graph
solution.save_solution()
print()
print("Code to generate solution graph: \n")
print(solution.code_for_graph)

Geting LLM reply...
Got LLM reply.

Code to generate solution graph: 

import networkx as nx

G = nx.DiGraph()

# 1. Load Autism service provider CSV file
G.add_node("provider_csv_path", node_type="data", data_path="E:\\Research\\LLM-Geo\\Address_extraction\\ACE_providers_AGIS.csv", description="Autism service provider CSV file")
G.add_node("load_provider_csv", node_type="operation", description="Load Autism service provider CSV file")
G.add_edge("provider_csv_path", "load_provider_csv")

# 2. Extract address using helper function and ChatGPT
G.add_node("provider_data", node_type="data", description="Autism service providers and their website URLs")
G.add_edge("load_provider_csv", "provider_data")
G.add_node("provider_addresses", node_type="data", description="Extracted Autism service provider addresses")
G.add_node("extract_address", node_type="operation", description="Extract Autism service provider addresses from webpages using ChatGPT")
G.add_edge("provider_data", "extract_address"

## Execute code to generate the solution graphto generate the solution graph

In [208]:
exec(solution.code_for_graph)
solution_graph = solution.load_graph_file()

# Show the graph
G = nx.read_graphml(solution.graph_file)  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), solution.task_name + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)
# html_name

E:\Research\LLM-Geo\Address_extraction.html


## Generate prompts and code for operations (functions)

In [209]:
# with open(r'F:\Research\LLM-Geo\Resident_at_risk_counting\Resident_at_risk_counting.pkl', 'rb') as f:
#     solution = pickle.load(f)

In [210]:
operations = solution.get_LLM_responses_for_operations()
solution.save_solution()

# all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])
# print("All operation code: \n")
# print(all_operation_code_str)

1 / 4, load_provider_csv
Geting LLM reply...
Got LLM reply.
2 / 4, extract_address
Geting LLM reply...
Got LLM reply.
3 / 4, join_addresses
Geting LLM reply...
Got LLM reply.
4 / 4, save_to_csv
Geting LLM reply...
Got LLM reply.


In [211]:
all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])
exec(all_operation_code_str)
print(all_operation_code_str)

import pandas as pd

def load_provider_csv(provider_csv_path='E:\\Research\\LLM-Geo\\Address_extraction\\ACE_providers_AGIS.csv'):
    # Description: Load Autism service provider CSV file
    # provider_csv_path: The path to the Autism service provider CSV file
    provider_data = pd.read_csv(provider_csv_path)
    return provider_data
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import helper

def extract_address(provider_data):
    """
    Description: Extract Autism service provider addresses from webpages using ChatGPT
    provider_data: A DataFrame containing provider names and their website URLs
    Return: A DataFrame containing provider names, website URLs, and extracted addresses
    """
    
    def get_address_from_web_page(url):
        try:
            r = requests.get(url)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, 'html.parser')

            possible_links = [url] + [link.get('href') for link in so

## Generate prompts and code for assembly program

In [212]:
assembly_LLM_response = solution.get_LLM_assembly_response()
solution.assembly_LLM_response = assembly_LLM_response
solution.save_solution()

# print("Assembly code: \n")
# print(solution.code_for_assembly)

Geting LLM reply...
Got LLM reply.
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import helper

def load_provider_csv(provider_csv_path='E:\\Research\\LLM-Geo\\Address_extraction\\ACE_providers_AGIS.csv'):
    provider_data = pd.read_csv(provider_csv_path)
    return provider_data

def extract_address(provider_data):
    
    def get_address_from_web_page(url):
        try:
            r = requests.get(url)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, 'html.parser')

            possible_links = [url] + [link.get('href') for link in soup.find_all('a') if 'about' in link.get('href', '').lower() or 'contact' in link.get('href', '').lower()]

            for link in possible_links:
                soup2 = BeautifulSoup(requests.get(link).text, 'html.parser')
                prompt = 'Extract the address of the Autism service provider from the following web page text: ' + soup2.get_text()
                response = help

## Execute assembly code

In [213]:
all_code = all_operation_code_str + '\n' + solution.code_for_assembly
print(solution.code_for_assembly)

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import helper

def load_provider_csv(provider_csv_path='E:\\Research\\LLM-Geo\\Address_extraction\\ACE_providers_AGIS.csv'):
    provider_data = pd.read_csv(provider_csv_path)
    return provider_data

def extract_address(provider_data):
    
    def get_address_from_web_page(url):
        try:
            r = requests.get(url)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, 'html.parser')

            possible_links = [url] + [link.get('href') for link in soup.find_all('a') if 'about' in link.get('href', '').lower() or 'contact' in link.get('href', '').lower()]

            for link in possible_links:
                soup2 = BeautifulSoup(requests.get(link).text, 'html.parser')
                prompt = 'Extract the address of the Autism service provider from the following web page text: ' + soup2.get_text()
                response = helper.get_LLM_reply(prompt=prompt, mod

In [221]:
exec(all_code)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Geting LLM reply...


 10%|████████▎                                                                          | 1/10 [00:06<00:58,  6.47s/it]

Got LLM reply.
Geting LLM reply...


 20%|████████████████▌                                                                  | 2/10 [00:20<01:26, 10.87s/it]

...

Got LLM reply.
Geting LLM reply...


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:08<00:07,  7.02s/it]

Got LLM reply.


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:08<00:00,  6.89s/it]

Error while extracting address from https://www.breakingbarrierstherapy.com: 403 Client Error: Forbidden for url: https://www.breakingbarrierstherapy.com/


In [223]:
combined_data 

,Provider,Web Site,Address
0,Behavior Change and Associates,http://www.behaviorchange.com,Behavior Change & Associates\n160 East Lake Br...
1,Bluestem Center for Autism,http://www.bluestemcenterforautism.com,"Rochester:\n124 Elton Hills Ln NW\nRochester, ..."
2,Building Blocks Behavioral Services,http://buildingblocksbehavioralservices.com,NaN
3,Clinical Behavior Analysis,https://cbacares.com/,NaN
4,"AlphaBee, Inc",http://www.alphabee.com/,The web page text does not provide a specific ...
5,Autism New Jersey,http://www.autismnj.org,NaN
6,"BGF Performance Systems, LLC",http://www.bgftherapy.com,"BGF Children's Therapy\n6430 N. Central Ave, 6..."
7,Behavioral and Educational Strategies and Trai...,http://www.bestforautism.com/,"2630 West Rumble Road, Modesto, CA. 95350"
8,"Beyond Expectations, Inc.",http://www.beitherapy.com,"Address: 15 10th Avenue Shalimar Florida, 32579"
9,Breaking Barriers Therapy Services,https://www.breakingbarrierstherapy.com,NaN


In [220]:
print(provider_addresses['Address'])

0    Behavior Change & Associates\n160 East Lake Br...
1    Rochester:\n124 Elton Hills Ln NW\nRochester, ...
2    The address of the Autism service provider, Al...
3    BGF Children's Therapy\n6430 N. Central Ave, 6...
4    The address of the Autism service provider is:...
5    Address of the Autism service provider:\n\nAdm...
Name: Address, dtype: object


In [216]:
address_components

[]

In [217]:
provider_n_website_columns = load_ACE_providers_csv()
raw_addresses = fetch_addresses_from_websites(provider_n_website_columns)
address_components = extract_address_components(raw_addresses)
output_csv = save_extracted_addresses(provider_n_website_columns, address_components)
print(f"Extracted addresses saved to {output_csv}")

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Geting LLM reply...
Got LLM reply.
Geting LLM reply...
Got LLM reply.
Geting LLM reply...


 10%|████████▎                                                                          | 1/10 [00:08<01:13,  8.22s/it]

Got LLM reply.
Geting LLM reply...
Got LLM reply.
Geting LLM reply...
Got LLM reply.
Geting LLM reply...
Got LLM reply.
Geting LLM reply...
Got LLM reply.


 20%|████████████████▌                                                                  | 2/10 [00:29<02:08, 16.00s/it]

...

Got LLM reply.
Error extracting address components for Behavior Change and Associates: not enough values to unpack (expected 4, got 1)
Geting LLM reply...


 13%|██████████▊                                                                        | 3/23 [00:06<00:43,  2.18s/it]

Got LLM reply.
Error extracting address components for Behavior Change and Associates: not enough values to unpack (expected 4, got 1)
Geting LLM reply...


 13%|██████████▊                                                                        | 3/23 [00:19<02:11,  6.56s/it]


KeyboardInterrupt: 

In [ ]:
stop

In [ ]:
# # Case 2: mobility 
# import geopandas as gpd

# def load_france_shp(france_shp_url="https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip"):
#     """
#     Description: Load France administrative regions shapefile

#     Args:
#     france_shp_url (str): URL of the zipped shapefile

#     Returns:
#     france_gdf (GeoDataFrame): GeoDataFrame containing the France administrative regions
#     """
#     france_gdf = gpd.read_file(france_shp_url)
#     return france_gdf
# import pandas as pd
# import geopandas as gpd
# import requests
# from io import StringIO

# def load_mobility_data(mobility_api_url="http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020"):
#     """
#     Load COVID-19 mobility data

#     mobility_api_url: REST API url for COVID-19 mobility data
#     returns: mobility_data (dataframe): Mobility data with place, date and intra_movement values
#     """
#     response = requests.get(mobility_api_url)
#     data = StringIO(response.text)
#     mobility_data = pd.read_csv(data)

#     return mobility_data
# def join_data(france_gdf=gpd.GeoDataFrame(), mobility_data=pd.DataFrame()):
#     """
#     Description: Join shapefile and mobility data

#     Args:
#     france_gdf (GeoDataFrame): GeoDataFrame containing the France administrative regions
#     mobility_data (pd.DataFrame): Mobility data with place, date, and intra_movement values

#     Returns:
#     joined_data (GeoDataFrame): GeoDataFrame containing the joined shapefile and mobility data
#     """
#     # Convert GID_1 column to string type without leading zeros
#     france_gdf["GID_1"] = france_gdf["GID_1"].astype(str).str.lstrip("0")

#     # Convert place column to string type without leading zeros
#     mobility_data["place"] = mobility_data["place"].astype(str).str.lstrip("0")

#     # Merge mobility data with shapefile based on GID_1 and place columns
#     joined_data = france_gdf.merge(mobility_data, left_on="GID_1", right_on="place")

#     # Remove duplicates
#     joined_data.drop_duplicates(subset=["GID_1", "date"], inplace=True)

#     return joined_data
# def calc_monthly_changes(joined_data=gpd.GeoDataFrame()):
#     """
#     Create monthly change rates
    
#     Args:
#     joined_data (gpd.GeoDataFrame): GeoDataFrame containing the joined shapefile and mobility data
    
#     Returns:
#     monthly_changes (pd.DataFrame): DataFrame with monthly change rates for each administrative region
#     """
#     # Parse dates and create a month-year column
#     joined_data['date'] = pd.to_datetime(joined_data['date'])
#     joined_data['month_year'] = joined_data['date'].dt.to_period('M')

#     # Calculate the sum of intra_movement for each region and month-year
#     region_monthly = joined_data.groupby(['NAME_1', 'month_year'])['intra_movement'].sum().reset_index()

#     # Calculate the changes of each month based on January 2020
#     base_month =  "2020-01" # pd.to_datetime("2020-01")
#     print(region_monthly[region_monthly['month_year'] == "2020-01"])
#     baseline = region_monthly[region_monthly['month_year'] == base_month].set_index('NAME_1')['intra_movement']
#     print(baseline)
#     region_monthly['change_rate'] = region_monthly.apply(lambda row: (row['intra_movement'] - baseline[row['NAME_1']]) / baseline[row['NAME_1']], axis=1)

#     # Pivot month_year as columns to create monthly_changes DataFrame
#     monthly_changes = region_monthly.pivot(index='NAME_1', columns='month_year', values='change_rate').reset_index()

#     return monthly_changes
# import matplotlib.pyplot as plt
# import seaborn as sns

# def matrix_map(monthly_changes=pd.DataFrame(), france_gdf=gpd.GeoDataFrame()):
#     """
#     Description: Create matrix of France maps

#     Args:
#     monthly_changes (pd.DataFrame): DataFrame with monthly change rates for each administrative region
#     france_gdf (gpd.GeoDataFrame): GeoDataFrame containing the France administrative regions

#     Returns:
#     france_map_fig (Figure): Figure containing the matrix of France maps with monthly change rates
#     """
#     # Merge monthly_changes with france_gdf
#     france_gdf = france_gdf.merge(monthly_changes, on="NAME_1")
    
#     # Set up plot
#     plt.style.use('seaborn-darkgrid')
#     france_map_fig, axs = plt.subplots(3, 4, figsize=(24, 18), sharex='col', sharey='row')
#     france_map_fig.suptitle('Monthly change rates of each administrative region in France')

#     # Iterate through columns for each month
#     for idx, month_year in enumerate(monthly_changes.columns[1:]):
#         row, col = divmod(idx, 4)
        
#         # Create subplot
#         france_gdf.plot(column=month_year, ax=axs[row, col], legend=True, cmap='coolwarm', legend_kwds={'shrink': 0.8})
#         axs[row, col].set_title(month_year.strftime("%B %Y"))
#         axs[row, col].set_xticks([])
#         axs[row, col].set_yticks([])
#         axs[row, col].set_xlabel("")
#         axs[row, col].set_ylabel("")
        
#     plt.tight_layout()
#     plt.subplots_adjust(hspace=0.05, wspace=0.05)
    
#     return france_map_fig
# import matplotlib.pyplot as plt
# import seaborn as sns

# def chart_matrix(monthly_changes=pd.DataFrame()):
#     """
#     Description: Create chart matrix of each administrative region's line chart

#     Args:
#     monthly_changes (pd.DataFrame): DataFrame with monthly change rates for each administrative region

#     Returns:
#     line_chart_matrix_fig (matplotlib.figure.Figure): Figure object containing the line chart matrix
#     """
#     # Set default seaborn style
#     sns.set()

#     # Prepare monthly_changes data
#     monthly_changes_tidy = pd.melt(monthly_changes, 
#                                    id_vars=["NAME_1"], 
#                                    var_name="month_year", 
#                                    value_name="change_rate")
#     monthly_changes_tidy["month_year"] = monthly_changes_tidy["month_year"].astype(str)

#     # Calculate the number of rows and columns for the line chart matrix
#     n_regions = len(monthly_changes["NAME_1"].unique())
#     n_cols = min(4, n_regions)
#     n_rows = (n_regions // n_cols) + (1 if (n_regions % n_cols) else 0)

#     # Create the line chart matrix
#     line_chart_matrix_fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5), sharex=True, sharey=True)

#     for idx, (name, group) in enumerate(monthly_changes_tidy.groupby(["NAME_1"])):
#         row, col = divmod(idx, n_cols)

#         ax = axes[row][col]
#         sns.lineplot(x="month_year", y="change_rate", data=group, ax=ax)
#         ax.set_title(name)
#         ax.set_xticklabels(labels=group["month_year"].unique(), rotation=45)

#     # Remove empty subplots
#     for idx in range(n_regions, n_rows * n_cols):
#         row, col = divmod(idx, n_cols)
#         axes[row][col].remove()

#     # Adjust layout
#     line_chart_matrix_fig.tight_layout()

#     return line_chart_matrix_fig
# import matplotlib.pyplot as plt

# def overall_trends_line_chart(monthly_changes=pd.DataFrame()):
#     """
#     Description: Draw the line chart for all administrative regions

#     Args:
#     monthly_changes (pd.DataFrame): DataFrame with monthly change rates for each administrative region

#     Returns:
#     trends_line_chart_fig (matplotlib.figure.Figure): Figure containing the line chart for all administrative regions
#     """
#     # Transpose monthly_changes DataFrame
#     transposed_changes = monthly_changes.set_index("NAME_1").transpose()

#     # Create line chart
#     trends_line_chart_fig, ax = plt.subplots(figsize=(12, 8))
#     transposed_changes.plot(ax=ax)
#     plt.xlabel("Month")
#     plt.ylabel("Change Rate")
#     plt.title("Monthly Trends of Change Rates for All Administrative Regions")
#     plt.legend(title="Regions", labels=transposed_changes.columns, bbox_to_anchor=(1.17, 1))
#     plt.xticks(range(len(transposed_changes.index)), transposed_changes.index, rotation=90)

#     return trends_line_chart_fig

# # Example usage
# # france_gdf = load_france_shp()
# # mobility_data = load_mobility_data()
# # joined_data = join_data(france_gdf, mobility_data)
# # monthly_changes = calc_monthly_changes(joined_data)
# # trends_line_chart_fig = overall_trends_line_chart(monthly_changes)
# # plt.show()
# import geopandas as gpd
# import pandas as pd
# import requests
# from io import StringIO
# import matplotlib.pyplot as plt
# import seaborn as sns


 
# # Load data
# monthly_changes = calc_monthly_changes(joined_data)

# # Create and save monthly change maps
# france_map_fig = matrix_map(monthly_changes, france_gdf)
# france_map_fig.savefig('france_monthly_change_maps.png')

# # Create and save chart matrix with line charts of each administrative region
# line_chart_matrix_fig = chart_matrix(monthly_changes)
# line_chart_matrix_fig.savefig('chart_matrix_line_charts.png')

# # Create and save overall trends line chart
# trends_line_chart_fig = overall_trends_line_chart(monthly_changes)
# trends_line_chart_fig.savefig('overall_trends_line_chart.png')

# plt.show()


In [ ]:
joined_data#.plot()